# Import Stuff & Setup

In [6]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary
import warnings
import os
import sys
from tqdm import tqdm
import cv2


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
RESOLUTION = 64
block_frame = 9
frame_interval = 15

In [10]:
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

In [11]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.Lambda(lambda x: x / 255.),
    transforms.Resize((RESOLUTION, RESOLUTION))
])


# Utils

In [7]:
def plot_image_from_list(__images, __labels, __count):
    plt.figure(figsize=(10, 10))
    for _i in range(__count[0] * __count[1]):
        plt.subplot(__count[0], __count[1], _i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        _img = __images[_i].to("cpu").numpy()
        plt.imshow(_img, cmap= "gray")
        plt.xlabel(__labels[_i])
    plt.show()

In [8]:
def plot_randomly_form_dataset(__dataset):
    images = []
    labels = []
    for _ in range(0, 25):
        index = torch.randint(0, len(__dataset), (1,)).item()
        image, label = __dataset[index]
        images.append(image[:,0].squeeze())
        labels.append(__dataset.label_id2str(label.item()))
    
    plot_image_from_list(images, labels, (5, 5))
        

In [9]:
def plot_sequence_form_dataset(__dataset, index):
    images = []
    labels = []
    for i in range(0, block_frame):
        image, label = __dataset[index]
        images.append(image[:,i].squeeze())
        labels.append(__dataset.label_id2str(label.item()))
    
    plot_image_from_list(images, labels, (1, block_frame))
        

In [10]:
def plot_randomly_form_dataset_model(__dataset, __model):
    images = []
    labels = []
    for _ in range(0, 25):
        index = torch.randint(0, len(__dataset), (1,)).item()
        image, label = __dataset[index]
        result = model(image.unsqueeze(0).to(device)).cpu().argmax()
        images.append(image[:,0].squeeze())
        labels.append(__dataset.label_id2str(label.item()) + " -> " + __dataset.label_id2str(result.item()))
    
    plot_image_from_list(images, labels, (5, 5))
        

# Dataset

In [12]:
# DATA_SOURCE = {
#     "Abuse": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Abuse",
#     "Arrest": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arrest",
#     "Arson": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arson",
#     "Assault": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Assault",
#     "Burglary": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Burglary",
#     "Explosion": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Explosion",
#     "Fighting": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2\Fighting",
#     "Normal" : "E:\\kaggle\\input\\crimeufcdataset\\Anomaly_Dataset\\Anomaly_Videos\\Normal-Videos-Part-1"
# }

DATA_SOURCE = {
    "Abuse": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-1\Anomaly-Videos-Part-1\Abuse",
    "Arrest": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-1\Anomaly-Videos-Part-1\Arrest",
    "Arson": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-1\Anomaly-Videos-Part-1\Arson",
    "Assault": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-1\Anomaly-Videos-Part-1\Assault",
    "Burglary": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-2\Anomaly-Videos-Part-2\Burglary",
    "Explosion": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-2\Anomaly-Videos-Part-2\Explosion",
    "Fighting": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-2\Anomaly-Videos-Part-2\Fighting",   
    "RoadAccidents":r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-3\Anomaly-Videos-Part-3\RoadAccidents",
    "Robbery": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-3\Anomaly-Videos-Part-3\Robbery",
    "Shooting": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-3\Anomaly-Videos-Part-3\Shooting",
    "Shoplifting": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-4\Anomaly-Videos-Part-4\Shoplifting",
    "Stealing": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-4\Anomaly-Videos-Part-4\Stealing",
    "Vandalism": r"D:\Anomaly-Detection-Dataset\Anomaly-Videos-Part-4\Anomaly-Videos-Part-4\Vandalism",
    "Normal": r"D:\Anomaly-Detection-Dataset\Training-Normal-Videos-Part-1\Training-Normal-Videos-Part-1"
   
}

TEST_SOURCE = {
    "Normal" : r"E:\kaggle\input\Anomaly-Detection-Dataset\Testing_Normal_Videos_Anomaly"
}

HEAD_PATH = "E:\\kaggle\\input\\Anomaly-Detection-Dataset"

# DATA_SOURCE = {
#     "Abuse": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Abusetest",
#     "Arrest": "E:\kaggle\input\crimeufcdataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1\Arresttest",
# }



In [13]:
import os
for directory_path in TEST_SOURCE.items():
# 경로가 존재하는지 확인합니다.
    if os.path.exists(directory_path[1]):
        print("경로가 존재합니다.")
        # 디렉터리 내의 파일 목록을 출력합니다.
        print("디렉터리 내용:", os.listdir(directory_path[1]))
    else:
        print("지정된 경로를 찾을 수 없습니다.")


경로가 존재합니다.
디렉터리 내용: ['Normal_Videos_003_x264.mp4', 'Normal_Videos_006_x264.mp4', 'Normal_Videos_010_x264.mp4', 'Normal_Videos_014_x264.mp4', 'Normal_Videos_015_x264.mp4', 'Normal_Videos_018_x264.mp4', 'Normal_Videos_019_x264.mp4', 'Normal_Videos_024_x264.mp4', 'Normal_Videos_025_x264.mp4', 'Normal_Videos_027_x264.mp4', 'Normal_Videos_033_x264.mp4', 'Normal_Videos_034_x264.mp4', 'Normal_Videos_041_x264.mp4', 'Normal_Videos_042_x264.mp4', 'Normal_Videos_048_x264.mp4', 'Normal_Videos_050_x264.mp4', 'Normal_Videos_051_x264.mp4', 'Normal_Videos_056_x264.mp4', 'Normal_Videos_059_x264.mp4', 'Normal_Videos_063_x264.mp4', 'Normal_Videos_067_x264.mp4', 'Normal_Videos_070_x264.mp4', 'Normal_Videos_100_x264.mp4', 'Normal_Videos_129_x264.mp4', 'Normal_Videos_150_x264.mp4', 'Normal_Videos_168_x264.mp4', 'Normal_Videos_175_x264.mp4', 'Normal_Videos_182_x264.mp4', 'Normal_Videos_189_x264.mp4', 'Normal_Videos_196_x264.mp4', 'Normal_Videos_203_x264.mp4', 'Normal_Videos_210_x264.mp4', 'Normal_Videos_217_

# Dataset

In [25]:
import torch
from torch.utils.data import Dataset
import numpy as np
import os
import random

class Normal_Loader(Dataset):
    """
    is_train = 1 <- train, 0 <- test
    """
    def __init__(self, is_train=1, path='./UCF-Crime/', modality='TWO'):
        super(Normal_Loader, self).__init__()
        self.is_train = is_train
        self.modality = modality
        self.path = path
        if self.is_train == 1:
            data_list = os.path.join(path, 'train_normal.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
        else:
            data_list = os.path.join(path, 'test_normalv2.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
            random.shuffle(self.data_list)
            self.data_list = self.data_list[:-10]
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.is_train == 1:
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', self.data_list[idx][:-1]+'.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', self.data_list[idx][:-1]+'.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            if self.modality == 'RGB':
                return rgb_npy
            elif self.modality == 'FLOW':
                return flow_npy
            else:
                return concat_npy
        else:
            name, frames, gts = self.data_list[idx].split(' ')[0], int(self.data_list[idx].split(' ')[1]), int(self.data_list[idx].split(' ')[2][:-1])
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', name + '.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', name + '.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            if self.modality == 'RGB':
                return rgb_npy, gts, frames
            elif self.modality == 'FLOW':
                return flow_npy, gts, frames
            else:
                return concat_npy, gts, frames

class Anomaly_Loader(Dataset):
    """
    is_train = 1 <- train, 0 <- test
    """
    def __init__(self, is_train=1, path='./UCF-Crime/', modality='TWO'):
        super(Anomaly_Loader, self).__init__()
        self.is_train = is_train
        self.modality = modality
        self.path = path
        if self.is_train == 1:
            data_list = os.path.join(path, 'train_anomaly.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
        else:
            data_list = os.path.join(path, 'test_anomalyv2.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.is_train == 1:
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', self.data_list[idx][:-1]+'.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', self.data_list[idx][:-1]+'.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            if self.modality == 'RGB':
                return rgb_npy
            elif self.modality == 'FLOW':
                return flow_npy
            else:
                return concat_npy
        else:
            name, frames, gts = self.data_list[idx].split('|')[0], int(self.data_list[idx].split('|')[1]), self.data_list[idx].split('|')[2][1:-2].split(',')
            gts = [int(i) for i in gts]
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', name + '.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', name + '.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            if self.modality == 'RGB':
                return rgb_npy, gts, frames
            elif self.modality == 'FLOW':
                return flow_npy, gts, frames
            else:
                return concat_npy, gts, frames

if __name__ == '__main__':
    loader2 = Normal_Loader(is_train=0)
    print(len(loader2))
    #print(loader[1], loader2[1])


140


# FFC, Learner2

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F



class Learner2(nn.Module):
    def __init__(self, input_dim=2048, drop_p=0.0):
        super(Learner2, self).__init__()
        self.filter1 = nn.LayerNorm(input_dim)
        self.filter2 = nn.LayerNorm(input_dim)
        self.relu2 = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.relu = nn.PReLU()
        self.dropout = nn.Dropout(drop_p)

        self.fc1 = nn.Linear(input_dim, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 256)
        self.fc4 = nn.Linear(256, 32)
        self.fc5 = nn.Linear(32, 1)


    def forward(self, x):
        x1 = self.relu2(x)

        out = self.fc5(self.relu(self.fc4((self.relu(self.fc3(self.dropout(self.relu(self.fc2(self.relu(self.fc1(x)))))))))))
        out2 = self.fc5(self.relu(self.fc4(self.relu(self.fc3(self.dropout(self.relu(self.fc2(self.relu(self.fc1(x1))))))))))
        out = out + out2*0.2
        return torch.sigmoid(out) 





# Learner

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F



class Learner(nn.Module):
    def __init__(self, input_dim=2048, drop_p=0.0):
        super(Learner, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(drop_p),
            nn.Linear(512, 32),
            nn.ReLU(),
            nn.Dropout(drop_p),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.drop_p = drop_p
        self.weight_init()
        self.vars = nn.ParameterList()

        for i, param in enumerate(self.classifier.parameters()):
            self.vars.append(param)

    def weight_init(self):
        for layer in self.classifier:
            if type(layer) == nn.Linear:
                nn.init.xavier_normal_(layer.weight)

    def forward(self, x, vars=None):
        if vars is None:
            vars = self.vars
        x = F.linear(x, vars[0], vars[1])
        x = F.relu(x)
        x = F.dropout(x, self.drop_p, training=self.training)
        x = F.linear(x, vars[2], vars[3])
        x = F.dropout(x, self.drop_p, training=self.training)
        x = F.linear(x, vars[4], vars[5])
        return torch.sigmoid(x)

    def parameters(self):
        """
        override this function since initial parameters will return with a generator.
        :return:
        """
        return self.vars




# Loss

In [28]:
import torch
import torch.nn.functional as F

def MIL(y_pred, batch_size, is_transformer=0):
    loss = torch.tensor(0.).cuda()
    loss_intra = torch.tensor(0.).cuda()
    sparsity = torch.tensor(0.).cuda()
    smooth = torch.tensor(0.).cuda()
    if is_transformer==0:
        y_pred = y_pred.view(batch_size, -1)
    else:
        y_pred = torch.sigmoid(y_pred)

    for i in range(batch_size):
        anomaly_index = torch.randperm(30).cuda()
        normal_index = torch.randperm(30).cuda()

        y_anomaly = y_pred[i, :32][anomaly_index]
        y_normal  = y_pred[i, 32:][normal_index]

        y_anomaly_max = torch.max(y_anomaly) # anomaly
        y_anomaly_min = torch.min(y_anomaly)

        y_normal_max = torch.max(y_normal) # normal
        y_normal_min = torch.min(y_normal)

        loss += F.relu(1.-y_anomaly_max+y_normal_max)

        sparsity += torch.sum(y_anomaly)*0.00008
        smooth += torch.sum((y_pred[i,:31] - y_pred[i,1:32])**2)*0.00008
    loss = (loss+sparsity+smooth)/batch_size

    return loss


# Main

In [17]:
from torch.utils.data import DataLoader
import os
from sklearn import metrics
import argparse


parser = argparse.ArgumentParser(description='PyTorch MIL Training')
parser.add_argument('--lr', default=0.001, type=float, help='learning rate')
parser.add_argument('--w', default=0.0010000000474974513, type=float, help='weight_decay')
parser.add_argument('--modality', default='TWO', type=str, help='modality')
parser.add_argument('--input_dim', default=2048, type=int, help='input_dim')
parser.add_argument('--drop', default=0.6, type=float, help='dropout_rate')
parser.add_argument('--FFC', '-r', action='store_true',help='FFC')
args = parser.parse_args(args=[])

best_auc = 0

normal_train_dataset = Normal_Loader(is_train=1, modality=args.modality)
normal_test_dataset = Normal_Loader(is_train=0, modality=args.modality)

anomaly_train_dataset = Anomaly_Loader(is_train=1, modality=args.modality)
anomaly_test_dataset = Anomaly_Loader(is_train=0, modality=args.modality)

normal_train_loader = DataLoader(normal_train_dataset, batch_size=30, shuffle=True)
normal_test_loader = DataLoader(normal_test_dataset, batch_size=1, shuffle=True)

anomaly_train_loader = DataLoader(anomaly_train_dataset, batch_size=30, shuffle=True) 
anomaly_test_loader = DataLoader(anomaly_test_dataset, batch_size=1, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if args.FFC:
    model = Learner2(input_dim=args.input_dim, drop_p=args.drop).to(device)
else:
    model = Learner(input_dim=args.input_dim, drop_p=args.drop).to(device)

optimizer = torch.optim.Adagrad(model.parameters(), lr= args.lr, weight_decay=args.w)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50])
criterion = MIL

def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (normal_inputs, anomaly_inputs) in enumerate(zip(normal_train_loader, anomaly_train_loader)):
        inputs = torch.cat([anomaly_inputs, normal_inputs], dim=1)
        batch_size = inputs.shape[0]
        inputs = inputs.view(-1, inputs.size(-1)).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, batch_size)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print('loss = {}', train_loss/len(normal_train_loader))
    scheduler.step()

def test_abnormal(epoch):
    model.eval()
    global best_auc
    auc = 0
    with torch.no_grad():
        for i, (data, data2) in enumerate(zip(anomaly_test_loader, normal_test_loader)):
            inputs, gts, frames = data
            inputs = inputs.view(-1, inputs.size(-1)).to(torch.device('cuda'))
            score = model(inputs)
            score = score.cpu().detach().numpy()
            score_list = np.zeros(frames[0])
            step = np.round(np.linspace(0, frames[0]//16, 33))

            for j in range(32):
                score_list[int(step[j])*16:(int(step[j+1]))*16] = score[j]

            gt_list = np.zeros(frames[0])
            for k in range(len(gts)//2):
                s = gts[k*2]
                e = min(gts[k*2+1], frames)
                gt_list[s-1:e] = 1

            inputs2, gts2, frames2 = data2
            inputs2 = inputs2.view(-1, inputs2.size(-1)).to(torch.device('cuda'))
            score2 = model(inputs2)
            score2 = score2.cpu().detach().numpy()
            score_list2 = np.zeros(frames2[0])
            step2 = np.round(np.linspace(0, frames2[0]//16, 33))
            for kk in range(32):
                score_list2[int(step2[kk])*16:(int(step2[kk+1]))*16] = score2[kk]
            gt_list2 = np.zeros(frames2[0])
            score_list3 = np.concatenate((score_list, score_list2), axis=0)
            gt_list3 = np.concatenate((gt_list, gt_list2), axis=0)

            fpr, tpr, thresholds = metrics.roc_curve(gt_list3, score_list3, pos_label=1)
            auc += metrics.auc(fpr, tpr)

        print('auc = {}',auc/140) 

        if best_auc < auc/140:
            print('Saving..')
            state = {
                'net': model.state_dict(),
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/ckpt.pth')
            best_auc = auc/140

for epoch in range(0, 75):
    train(epoch)
    test_abnormal(epoch)




Epoch: 0
loss = {} 0.9601337114969889
auc = {} 0.8117433625611923
Saving..

Epoch: 1
loss = {} 0.8852749290289702
auc = {} 0.8181985799171346
Saving..

Epoch: 2
loss = {} 0.8277932052259092
auc = {} 0.8107278698114896

Epoch: 3
loss = {} 0.7856853979605215
auc = {} 0.8005859823942454

Epoch: 4
loss = {} 0.7388262550036112
auc = {} 0.8190237151962382
Saving..

Epoch: 5
loss = {} 0.7110931520108823
auc = {} 0.8189048280699037

Epoch: 6
loss = {} 0.6836954575997812
auc = {} 0.8217231828719435
Saving..

Epoch: 7
loss = {} 0.648902021072529
auc = {} 0.8165468275522304

Epoch: 8
loss = {} 0.6220153106583489
auc = {} 0.8131309210459872

Epoch: 9
loss = {} 0.6039476482956497
auc = {} 0.8111239564371283

Epoch: 10
loss = {} 0.5900650763953174
auc = {} 0.8237475302700968
Saving..

Epoch: 11
loss = {} 0.5636470593788006
auc = {} 0.8152787652409081

Epoch: 12
loss = {} 0.5489962178247946
auc = {} 0.8260400515765248
Saving..

Epoch: 13
loss = {} 0.5343993438614739
auc = {} 0.8260132756315776

Epoc

In [29]:
from torch.utils.data import DataLoader
import os
from sklearn import metrics
import argparse


parser = argparse.ArgumentParser(description='PyTorch MIL Training')
parser.add_argument('--lr', default=0.001, type=float, help='learning rate')
parser.add_argument('--w', default=0.0010000000474974513, type=float, help='weight_decay')
parser.add_argument('--modality', default='TWO', type=str, help='modality')
parser.add_argument('--input_dim', default=2048, type=int, help='input_dim')
parser.add_argument('--drop', default=0.6, type=float, help='dropout_rate')
parser.add_argument('--FFC', '-r', action='store_true',help='FFC')
args = parser.parse_args(args=[])

best_auc = 0

normal_test_dataset = Normal_Loader(is_train=0, modality=args.modality)

anomaly_test_dataset = Anomaly_Loader(is_train=0, modality=args.modality)

normal_test_loader = DataLoader(normal_test_dataset, batch_size=1, shuffle=True)

anomaly_test_loader = DataLoader(anomaly_test_dataset, batch_size=1, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


if args.FFC:
    model = Learner2(input_dim=args.input_dim, drop_p=args.drop).to(device)
else:
    model = Learner(input_dim=args.input_dim, drop_p=args.drop).to(device)

checkpoint = torch.load('./checkpoint/ckpt.pth')
print(checkpoint["net"])
model.load_state_dict(checkpoint['net'])

def test_abnormal():
    model.eval()
    global best_auc
    auc = 0
    with torch.no_grad():
        for i, (data, data2) in enumerate(zip(anomaly_test_loader, normal_test_loader)):
            inputs, gts, frames = data
            inputs = inputs.view(-1, inputs.size(-1)).to(torch.device('cuda'))
            score = model(inputs)
            score = score.cpu().detach().numpy()
            score_list = np.zeros(frames[0])
            step = np.round(np.linspace(0, frames[0]//16, 33))

            for j in range(32):
                score_list[int(step[j])*16:(int(step[j+1]))*16] = score[j]

            gt_list = np.zeros(frames[0])
            for k in range(len(gts)//2):
                s = gts[k*2]
                e = min(gts[k*2+1], frames)
                gt_list[s-1:e] = 1

            inputs2, gts2, frames2 = data2
            inputs2 = inputs2.view(-1, inputs2.size(-1)).to(torch.device('cuda'))
            score2 = model(inputs2)
            score2 = score2.cpu().detach().numpy()
            score_list2 = np.zeros(frames2[0])
            step2 = np.round(np.linspace(0, frames2[0]//16, 33))
            for kk in range(32):
                score_list2[int(step2[kk])*16:(int(step2[kk+1]))*16] = score2[kk]
            gt_list2 = np.zeros(frames2[0])
            score_list3 = np.concatenate((score_list, score_list2), axis=0)
            gt_list3 = np.concatenate((gt_list, gt_list2), axis=0)

            fpr, tpr, thresholds = metrics.roc_curve(gt_list3, score_list3, pos_label=1)
            auc += metrics.auc(fpr, tpr)

        print('auc = {}',auc/140) 

        if best_auc < auc/140:
            print('Saving..')
            state = {
                'net': model.state_dict(),
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/ckpt.pth')
            best_auc = auc/140

test_abnormal()



OrderedDict([('classifier.0.weight', tensor([[-1.9256e-03,  3.2345e-02, -3.5038e-02,  ..., -2.5452e-02,
         -3.7166e-02,  1.8154e-02],
        [ 7.3668e-03,  3.2023e-04,  2.6721e-05,  ...,  2.8895e-04,
          2.9877e-06, -1.9084e-05],
        [-2.7635e-02,  1.9361e-03, -2.8754e-02,  ...,  1.4045e-03,
         -1.2228e-02,  2.4967e-02],
        ...,
        [-2.5875e-02, -3.4827e-03,  4.2782e-03,  ..., -7.4396e-04,
          3.7952e-02, -2.8939e-03],
        [ 2.1114e-03, -4.3645e-03, -1.6992e-02,  ...,  3.0510e-03,
         -1.0844e-03,  5.9644e-03],
        [ 4.9684e-07,  1.1315e-07,  3.3928e-05,  ..., -1.9603e-02,
          1.2675e-04, -6.4794e-08]], device='cuda:0')), ('classifier.0.bias', tensor([ 7.9294e-03, -1.3795e-02,  1.8329e-02, -6.3392e-03,  5.6112e-03,
         2.4824e-02, -1.0557e-02,  2.0121e-03,  2.0255e-02,  4.4309e-03,
         1.0054e-02, -6.5135e-03,  1.7570e-02, -1.1332e-02, -1.4355e-02,
         1.7141e-02,  2.1839e-03, -1.1565e-04,  1.1847e-02,  1.8383e-03